# IMPORTS

In [3]:
import pandas as pd, numpy as np
import sklearn
import warnings
warnings.filterwarnings('ignore')

from ipynb.fs.full.Data_Preprocessing import remove_stop_words, make_feature_vec

# TRAIN-TEST SPLIT + PROCESSING

In [4]:
data = pd.read_csv('../data/train.csv')
labels=(data["toxic"] | data["severe_toxic"] | data["obscene"] | data["threat"] | data["insult"] | data["identity_hate"])

train_data = data[0:100000]
train_labels = labels[0:100000]
test_data = data[100001:]
test_labels = labels[100001:]

### CLEANING DATA 
clean_comments = remove_stop_words(data)
ret = make_feature_vec(clean_comments)

train_feature_vectors_sparse = ret['train_feature_vectors_sparse'][0:100000]
test_feature_vectors_sparse = ret['train_feature_vectors_sparse'][100001:]
vocab = ret['vocab']

6.26680286518227% of 100.0% processed
12.53360573036454% of 100.0% processed
18.80040859554681% of 100.0% processed
25.06721146072908% of 100.0% processed
31.33401432591135% of 100.0% processed
37.60081719109362% of 100.0% processed
43.86762005627589% of 100.0% processed
50.13442292145816% of 100.0% processed
56.40122578664043% of 100.0% processed
62.6680286518227% of 100.0% processed
68.93483151700497% of 100.0% processed
75.20163438218724% of 100.0% processed
81.46843724736951% of 100.0% processed
87.73524011255178% of 100.0% processed
94.00204297773405% of 100.0% processed
Data processed!
Starting to create bag of words...
Bag of words created.
Vocabulary size:168595


In [5]:
print("Shape of train vec:", train_feature_vectors_sparse.shape)
print("Shape of test vec:", test_feature_vectors_sparse.shape)

Shape of train vec: (100000, 168595)
Shape of test vec: (59570, 168595)


# TRAINING NB CLASSIFIER

In [6]:
from sklearn.naive_bayes import ComplementNB
clf = ComplementNB()
clf.fit(train_feature_vectors_sparse, train_labels)

ComplementNB(alpha=1.0, class_prior=None, fit_prior=True, norm=False)

###### TEST DATA PROCESSING

In [7]:
##### STRAY BLOCK FOR LATER USAGE #####

# # creating sparse representation of test feature vectors
# vocab_mapping=set(zip([i for i in range (0,len(vocab))],vocab))
# print("Starting to create bag of words...")
# vectorizer_test = CountVectorizer(analyzer="word", tokenizer=None, preprocessor=None, vocabulary=vocab_mapping)
# test_feature_vectors_sparse = vectorizer_test.fit_transform(clean_test_comments)
# test_feature_vectors_full = test_feature_vectors_sparse.toarray()

# print("Bag of words created.")

# PREDICTION

In [8]:
predictions = clf.predict(test_feature_vectors_sparse)

print(sum(test_labels)/len(test_labels))
print(sum(train_labels)/len(train_labels))

0.101292596945
0.10191


# PERFORMANCE METRICS

In [9]:
confmat = sklearn.metrics.confusion_matrix(test_labels, predictions)

In [10]:
precision = confmat[1,1]/(confmat[1,1] + confmat[1,0])
recall = confmat[1,1]/(confmat[1,1] + confmat[0,1])
fScore = (2*precision*recall)/(recall+precision)
ccr = (confmat[0,0] + confmat[1,1])/(sum(sum(confmat)))

print("Precision = ",precision)
print("Recall = ",recall)
print("F_score = ",fScore)
print("CCR = ",ccr)

Precision =  0.712462711303
Recall =  0.588904109589
F_score =  0.644817759112
CCR =  0.92049689441


In [11]:
confmat

array([[50535,  3001],
       [ 1735,  4299]])

In [12]:
len(vocab)

168595